In [20]:
%load_ext autoreload
%autoreload 2

from train import *
from models import *
from tools import load_data, get_label_dict, get_word_labels

from keras import optimizers
from keras.models import load_model
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

from PIL import Image
from IPython.display import display

import numpy as np
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
X_train, y_train = load_data()
X_val, y_val = load_data("val")
X_test, _ = load_data("test")

y_train = to_categorical(y_train, 200)
y_val = to_categorical(y_val, 200)

Loading train data

Finished loading train data

Loading val data

Finished loading val data

Loading test data

Finished loading test data



In [22]:
labels = get_label_dict()
words = get_word_labels()

test_images = os.listdir("data/tiny-imagenet-200/test/images/")
assert len(X_test) == len(test_images)

Loading labels

Done

Loading words

Done



In [23]:
# Image preprocessing for the data
datagen = ImageDataGenerator(width_shift_range=0.2,
                             height_shift_range=0.2, 
                             channel_shift_range=115., 
                             fill_mode="reflect", 
                             preprocessing_function=shuffle_channels, 
                             rescale=1./255)

testgen = ImageDataGenerator(rescale=1./255)

data = {"train": (X_train, y_train), 
        "val": (X_val, y_val)}

In [24]:
model = AlphaNet()
model_prefix = "models/" + model.name + "/" + model.name
model_path = model_prefix + ".h5"

In [25]:
# Checkpoint callback to only save best model
checkpoint = ModelCheckpoint(model_path, 
                             monitor="val_accuracy", 
                             verbose=1, 
                             save_best_only=True, 
                             mode="auto", 
                             period=1)

cb_list = [checkpoint]

In [ ]:
optim = optimizers.Adam(learning_rate=0.001)

history = train(model_path=model_path, 
                restore=False, 
                epochs=300,
                model=model, 
                optim=optim,
                datagen=datagen,
                data=data, 
                cb_list=cb_list)

Model: "AlphaNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gaussian_noise_3 (GaussianNo (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 64, 64, 32)        4736      
_________________________________________________________________
batch_normalization_15 (Batc (None, 64, 64, 32)        128       
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
spatial_dropout2d_9 (Spatial (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 32, 32, 64)        512

3125/3125 [==============================] - 165s 53ms/step - loss: 4.8470 - accuracy: 0.0438 - top3_accuracy: 0.1020 - top5_accuracy: 0.1472 - val_loss: 5.1363 - val_accuracy: 0.0510 - val_top3_accuracy: 0.1133 - val_top5_accuracy: 0.1598

Epoch 00010: val_accuracy improved from 0.04820 to 0.05100, saving model to models/AlphaNet/AlphaNet.h5
Epoch 11/300
3125/3125 [==============================] - 165s 53ms/step - loss: 4.8128 - accuracy: 0.0470 - top3_accuracy: 0.1095 - top5_accuracy: 0.1567 - val_loss: 4.9634 - val_accuracy: 0.0493 - val_top3_accuracy: 0.1149 - val_top5_accuracy: 0.1635

Epoch 00011: val_accuracy did not improve from 0.05100
Epoch 12/300
3125/3125 [==============================] - 164s 52ms/step - loss: 4.7830 - accuracy: 0.0499 - top3_accuracy: 0.1146 - top5_accuracy: 0.1630 - val_loss: 4.5714 - val_accuracy: 0.0552 - val_top3_accuracy: 0.1243 - val_top5_accuracy: 0.1738

Epoch 00012: val_accuracy improved from 0.05100 to 0.05520, saving model to models/AlphaNet/

In [ ]:
show_history(history)

In [ ]:
# This loads the model and makes predictions on the test set
model = load_model(model_path)
output = model.predict_generator(testgen.flow(X_test, shuffle=False))
results = []

for result in output:
    results.append(labels[np.argmax(result)])
    
assert len(results) == len(X_test)
print("Test images predicted:", len(results))

In [ ]:
# This writes the test set predictions to file
with open(model_prefix + " Predictions.txt", "w") as test_file:
    for i in range(len(results)):
        test_file.write(test_images[i] + " " + results[i] + "\n")
    print("Predictions saved at", test_file.name)

In [ ]:
# This shows some test set images and the model's predictions on them
display_dim = 128
for _ in range(10):
    index = np.random.randint(len(X_test))
    img = Image.fromarray(X_test[index], "RGB").resize(size=(display_dim, display_dim))
    word_label = words[results[index]]
    display(img)
    print(word_label)

In [ ]:
# This saves an HTML version of the notebook for later viewing
os.system("jupyter nbconvert --to html Training.ipynb")

html_path = model_prefix + ".html"
if path.exists(html_path):
    os.remove(html_path)
os.rename("Training.html", html_path)